## Data Aggregation and Group Operations
- GroupBy Mechanics
- Data Aggregation
- Apply: General `split-apply-combine`
- Pivot Table and Cross Tabulation

In [235]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## 1 | GroupBy Mechanism

In [236]:
df = pd.DataFrame(
    {
        "key1" : ["a", "a", None, "b", "b", "a", None],
        "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
        "data1" : np.random.standard_normal(7),
        "data2" : np.random.standard_normal(7)
    }
)
df

,key1,key2,data1,data2
0,a,1,-1.085164,1.500688
1,a,2,-0.004205,0.689721
2,None,1,-0.454738,-0.142520
3,b,2,-0.133175,0.454850
4,b,1,0.809192,0.193644
5,a,<NA>,0.350124,-0.405043
6,None,1,-0.585485,-0.380360


In [237]:
grouped = df["data1"].groupby(df["key1"]) # dropna default True
grouped

In [238]:
grouped.mean()

key1
a   -0.246415
b    0.338008
Name: data1, dtype: float64

In [239]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means # Group By using multiple columns

key1  key2
a     1      -1.085164
      2      -0.004205
b     1       0.809192
      2      -0.133175
Name: data1, dtype: float64

In [240]:
means.unstack()

key2,1,2
key1,,
a,-1.085164,-0.004205
b,0.809192,-0.133175


📌 `Note` - The length of both the arrays must be same as no.rows in dataset

In [241]:
# Group by with external columns
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]
df["data1"].groupby([states, years]).mean()

CA  2005    0.172960
    2006   -0.454738
OH  2005   -0.609169
    2006    0.111854
Name: data1, dtype: float64

In [242]:
df.groupby("key1").mean()
df.groupby("key2").mean(numeric_only=True) # Only numeric dtype columns

,data1,data2
key2,,
1,-0.329049,0.292863
2,-0.068690,0.572285


In [243]:
df.groupby(["key1", "key2"]).size() # Number of Rows in that group including Na values

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [244]:
df.groupby("key1", dropna=False).size()
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [245]:
df.groupby("key1").count() # Number of values in that column excluding Na values

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


### | Iterating Over Groups

In [246]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -1.085164  1.500688
1    a     2 -0.004205  0.689721
5    a  <NA>  0.350124 -0.405043
b
  key1  key2     data1     data2
3    b     2 -0.133175  0.454850
4    b     1  0.809192  0.193644


📌 `Note` - Applying GroupBy using more than one columns, its just the unique combinations of values of all these columns

In [247]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', 1)
  key1  key2     data1     data2
0    a     1 -1.085164  1.500688
('a', 2)
  key1  key2     data1     data2
1    a     2 -0.004205  0.689721
('b', 1)
  key1  key2     data1     data2
4    b     1  0.809192  0.193644
('b', 2)
  key1  key2     data1    data2
3    b     2 -0.133175  0.45485


📌 `Note` - By storing data like this you can perform analysis between different categories data

In [248]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"] # Dictonary Comprehensions

,key1,key2,data1,data2
3,b,2,-0.133175,0.454850
4,b,1,0.809192,0.193644


### | Grouping on Columns

In [249]:
grouped = df.T.groupby(
    {
        # Since all column names are distinct you can reframe them like this
        # actual_column_name : new_referance_category
        "key1": "key", 
        "key2": "key",
        "data1": "data", 
        "data2": "data"
    }
)

col_groupby = {
    name : group for name, group in grouped
}
col_groupby['key']

,0,1,2,3,4,5,6
key1,a,a,None,b,b,a,None
key2,1,2,1,2,1,<NA>,1


In [250]:
# Same above implementation for index groupby
grouped = df.set_index(['key1']).groupby(
    {
        'a': 'cat1',
        'b': 'cat2',
        None: 'cat3'
    }
)

col_groupby = {
    name : group for name, group in grouped
}
col_groupby['cat3']

,key2,data1,data2
key1,,,
None,1,-0.454738,-0.14252
None,1,-0.585485,-0.38036


### | Selecting Column or Subset of Columns

In [251]:
# Fetching a perticular column
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     1.500688
     2     0.689721
b    1     0.193644
     2     0.454850

In [252]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped # Returns an iterator
s_grouped.mean()

key1  key2
a     1       1.500688
      2       0.689721
b     1       0.193644
      2       0.454850
Name: data2, dtype: float64

### | Grouping with Dicts and Series

In [253]:
people = pd.DataFrame(
    np.random.standard_normal((5, 5)),
    columns=["a", "b", "c", "d", "e"],
    index=["Joe", "Steve", "Wanda", "Jill", "Trey"]
)

people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,0.293126,-0.811371,0.537303,2.197960,-2.088296
Steve,1.257486,0.021169,-0.712066,1.208125,-2.539238
Wanda,-0.232088,NaN,NaN,-0.773131,0.491882
Jill,-2.090672,-0.838393,0.429137,0.226021,0.861542
Trey,-0.123496,1.661396,0.031116,1.531401,-0.103718


In [254]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}

In [255]:
# Mapping with Dictonary
by_column = people.T.groupby(mapping)
by_column.sum()

,Joe,Steve,Wanda,Jill,Trey
blue,2.735263,0.496059,-0.773131,0.655157,1.562517
red,-2.606541,-1.260583,0.259794,-2.067524,1.434182


In [256]:
# Mapping with Series
map_series = pd.Series(mapping)
map_series
people.T.groupby(map_series).count()

,Joe,Steve,Wanda,Jill,Trey
blue,2,2,1,2,2
red,3,3,2,3,3


### | Grouping with Functions
📌 `Note` - If you pass a function constructor ranther than a valid column name then the gruopby always performs over indices

In [257]:
people.groupby(by=None, level=0)
# You have to pass either by -> Column_name or level -> Index name or level

In [258]:
# Passing a function constructor
people.groupby(len).sum() # This is applied on length of indices

,a,b,c,d,e
3,0.293126,-0.811371,0.537303,2.197960,-2.088296
4,-2.214168,0.823002,0.460253,1.757421,0.757824
5,1.025398,0.021169,-0.712066,0.434994,-2.047357


In [259]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()
# With index and external column

,,a,b,c,d,e
3,one,0.293126,-0.811371,0.537303,2.197960,-2.088296
4,two,-2.090672,-0.838393,0.031116,0.226021,-0.103718
5,one,-0.232088,0.021169,-0.712066,-0.773131,-2.539238


### | Grouping by Index Level

In [260]:
columns = pd.MultiIndex.from_arrays(
    [["US", "US", "US", "JP", "JP"], [1, 3, 5, 1, 3]], names=["cty", "tenor"]
)
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.017230  1.294532  0.508221  1.033275 -1.557035
1      0.311465  0.318839 -0.628164  1.196036 -0.151036
2      0.862520 -1.180045  1.171173  1.214167  1.239844
3     -1.975707 -0.831697 -1.510720  0.790553 -0.899498

In [261]:
hier_df.T.groupby(level="cty").count()

,0,1,2,3
cty,,,,
JP,2,2,2,2
US,3,3,3,3


---

## 2 | Data Aggregation

In [262]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2) # 2nd Smallest from data1 for each group

key1   
a     0   -1.085164
      1   -0.004205
b     3   -0.133175
      4    0.809192
Name: data1, dtype: float64

📌 `Important Method` - agg(inbuilt_method/custom_function): To apply your custom Aggregate function

In [263]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.435287,1.905731
b,1,0.942367,0.261206


In [264]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0 -0.246415  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.338008  ...   

                        data2                                          \
          75%       max count      mean       std       min       25%   
key1                                                                    
a     0.17296  0.350124   3.0  0.595122  0.956381 -0.405043  0.142339   
b     0.57360  0.809192   2.0  0.324247  0.184701  0.193644  0.258945   

                                    
           50%       75%       max  
key1                                
a     0.689721  1.095204  1.500688  
b     0.324247  0.389549  0.454850  

[2 rows x 24 columns]

### | Column-wise and Multiple Function Application

In [265]:
import seaborn as sns
sns.get_dataset_names()
tips = sns.load_dataset(name='tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [266]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [267]:
grouped = tips.groupby(["day", "smoker"], observed=True) # Future Warning

In [268]:
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

day   smoker
Thur  Yes       0.163863
      No        0.160298
Fri   Yes       0.174783
      No        0.151650
Sat   Yes       0.147906
      No        0.158048
Sun   Yes       0.187250
      No        0.160113
Name: tip_pct, dtype: float64

| Passing a `list / Tuple / dictonary` of Multiple Functions

In [269]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Thur Yes     0.163863  0.039389      0.151240
     No      0.160298  0.038774      0.193350
Fri  Yes     0.174783  0.051293      0.159925
     No      0.151650  0.028123      0.067349
Sat  Yes     0.147906  0.061375      0.290095
     No      0.158048  0.039767      0.235193
Sun  Yes     0.187250  0.154134      0.644685
     No      0.160113  0.042347      0.193226

| Tuple - ('name_as_column', function)

In [270]:
grouped_pct.agg([("average", "mean"), ("stdev", "std")])

average     stdev
day  smoker                    
Thur Yes     0.163863  0.039389
     No      0.160298  0.038774
Fri  Yes     0.174783  0.051293
     No      0.151650  0.028123
Sat  Yes     0.147906  0.061375
     No      0.158048  0.039767
Sun  Yes     0.187250  0.154134
     No      0.160113  0.042347

In [271]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Thur Yes         17  0.163863  0.241255         17  19.190588  43.11
     No          45  0.160298  0.266312         45  17.113111  41.19
Fri  Yes         15  0.174783  0.263480         15  16.813333  40.17
     No           4  0.151650  0.187735          4  18.420000  22.75
Sat  Yes         42  0.147906  0.325733         42  21.276667  50.81
     No          45  0.158048  0.291990         45  19.661778  48.33
Sun  Yes         19  0.187250  0.710345         19  24.120000  45.35
     No          57  0.160113  0.252672         57  20.506667  48.17

In [272]:
result["tip_pct"]

count      mean       max
day  smoker                           
Thur Yes        17  0.163863  0.241255
     No         45  0.160298  0.266312
Fri  Yes        15  0.174783  0.263480
     No          4  0.151650  0.187735
Sat  Yes        42  0.147906  0.325733
     No         45  0.158048  0.291990
Sun  Yes        19  0.187250  0.710345
     No         57  0.160113  0.252672

In [273]:
ftuples = [("Average", "mean"), ("Variance", "var")]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Thur Yes     0.163863  0.001551  19.190588   69.808518
     No      0.160298  0.001503  17.113111   59.625081
Fri  Yes     0.174783  0.002631  16.813333   82.562438
     No      0.151650  0.000791  18.420000   25.596333
Sat  Yes     0.147906  0.003767  21.276667  101.387535
     No      0.158048  0.001581  19.661778   79.908965
Sun  Yes     0.187250  0.023757  24.120000  109.046044
     No      0.160113  0.001793  20.506667   66.099980

📌 `Important` - By passing dictonary you can maniupulate which Aggregate function wants to apply on different columns

In [274]:
grouped.agg({"tip" : "max", "size" : "sum"})
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"], "size" : "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Thur Yes     0.090014  0.241255  0.163863  0.039389   40
     No      0.072961  0.266312  0.160298  0.038774  112
Fri  Yes     0.103555  0.263480  0.174783  0.051293   31
     No      0.120385  0.187735  0.151650  0.028123    9
Sat  Yes     0.035638  0.325733  0.147906  0.061375  104
     No      0.056797  0.291990  0.158048  0.039767  115
Sun  Yes     0.065660  0.710345  0.187250  0.154134   49
     No      0.059447  0.252672  0.160113  0.042347  167

### | Returning Aggregated Data without row indexes

In [275]:
grouped = tips.groupby(["day", "smoker"], as_index=False, observed=True)
grouped.mean(numeric_only=True) # Same as applying reset index on grouped object

,day,smoker,total_bill,tip,size,tip_pct
0,Thur,Yes,19.190588,3.030000,2.352941,0.163863
1,Thur,No,17.113111,2.673778,2.488889,0.160298
2,Fri,Yes,16.813333,2.714000,2.066667,0.174783
3,Fri,No,18.420000,2.812500,2.250000,0.151650
4,Sat,Yes,21.276667,2.875476,2.476190,0.147906
5,Sat,No,19.661778,3.102889,2.555556,0.158048
6,Sun,Yes,24.120000,3.516842,2.578947,0.187250
7,Sun,No,20.506667,3.167895,2.929825,0.160113


---

## 3 | Apply: General split-apply-combine

In [276]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]
top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525


In [277]:
tips.groupby("smoker", observed=True).apply(top)
# DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
Yes    172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
No     232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746

In [278]:
tips.groupby("smoker", observed=True).apply(top, include_groups = False)

total_bill   tip     sex   day    time  size   tip_pct
smoker                                                            
Yes    172        7.25  5.15    Male   Sun  Dinner     2  0.710345
       178        9.60  4.00  Female   Sun  Dinner     2  0.416667
       67         3.07  1.00  Female   Sat  Dinner     1  0.325733
       183       23.17  6.50    Male   Sun  Dinner     4  0.280535
       109       14.31  4.00  Female   Sat  Dinner     2  0.279525
No     232       11.61  3.39    Male   Sat  Dinner     2  0.291990
       149        7.51  2.00    Male  Thur   Lunch     2  0.266312
       51        10.29  2.60  Female   Sun  Dinner     2  0.252672
       185       20.69  5.00    Male   Sun  Dinner     5  0.241663
       88        24.71  5.85    Male  Thur   Lunch     2  0.236746

In [279]:
tips.groupby(["smoker", "day"], observed=True).apply(top, n=1, column="total_bill", include_groups=False)

total_bill    tip     sex    time  size   tip_pct
smoker day                                                        
Yes    Thur 197       43.11   5.00  Female   Lunch     4  0.115982
       Fri  95        40.17   4.73    Male  Dinner     4  0.117750
       Sat  170       50.81  10.00    Male  Dinner     3  0.196812
       Sun  182       45.35   3.50    Male  Dinner     3  0.077178
No     Thur 142       41.19   5.00    Male   Lunch     5  0.121389
       Fri  94        22.75   3.25  Female  Dinner     2  0.142857
       Sat  212       48.33   9.00    Male  Dinner     4  0.186220
       Sun  156       48.17   5.00    Male  Dinner     6  0.103799

In [280]:
result = tips.groupby("smoker", observed=True)["tip_pct"].describe()
result
result.unstack("smoker")

       smoker
count  Yes        93.000000
       No        151.000000
mean   Yes         0.163196
       No          0.159328
std    Yes         0.085119
       No          0.039910
min    Yes         0.035638
       No          0.056797
25%    Yes         0.106771
       No          0.136906
50%    Yes         0.153846
       No          0.155625
75%    Yes         0.195059
       No          0.185014
max    Yes         0.710345
       No          0.291990
dtype: float64

### | Suppressing the Group Keys

In [281]:
tips.groupby("smoker", group_keys=False, observed = True).apply(top, include_groups = False)

,total_bill,tip,sex,day,time,size,tip_pct
172,7.25,5.15,Male,Sun,Dinner,2,0.710345
178,9.60,4.00,Female,Sun,Dinner,2,0.416667
67,3.07,1.00,Female,Sat,Dinner,1,0.325733
183,23.17,6.50,Male,Sun,Dinner,4,0.280535
109,14.31,4.00,Female,Sat,Dinner,2,0.279525
232,11.61,3.39,Male,Sat,Dinner,2,0.291990
149,7.51,2.00,Male,Thur,Lunch,2,0.266312
51,10.29,2.60,Female,Sun,Dinner,2,0.252672
185,20.69,5.00,Male,Sun,Dinner,5,0.241663
88,24.71,5.85,Male,Thur,Lunch,2,0.236746


### | Quantiles and Bucket Analysis

In [282]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,-0.380032,-2.073274
1,0.903999,1.682082
2,-0.875996,-0.129061
3,0.736154,0.548257
4,-0.389485,-0.647613


In [283]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(axis = 0), "max": group.max(axis = 0),
        "count": group.count(axis = 0), "mean": group.mean(axis = 0)}
    )

grouped = frame.groupby(quartiles, observed=True)
grouped.apply(get_stats)

# The reason behind the hierarchy of index as data1, data2 ..... because you are applying aggoperations on axis 0 which returns column wise values having column names as indexes

min       max  count      mean
data1                                           
1     data1 -1.821102  1.891992     10 -0.359810
      data2 -1.937895  1.670233     10  0.014589
2     data1 -3.378971  3.173246    345 -0.001132
      data2 -2.605846  2.592809    345  0.107480
3     data1 -3.500173  2.962216    579  0.032508
      data2 -4.401228  3.155617    579  0.082246
4     data1 -1.996116  2.160462     66  0.282592
      data2 -2.219057  1.834042     66 -0.374322

In [284]:
grouped.agg(["min", "max", "count", "mean"])

data1                               data2                          
            min       max count      mean       min       max count      mean
data1                                                                        
1     -1.821102  1.891992    10 -0.359810 -1.937895  1.670233    10  0.014589
2     -3.378971  3.173246   345 -0.001132 -2.605846  2.592809   345  0.107480
3     -3.500173  2.962216   579  0.032508 -4.401228  3.155617   579  0.082246
4     -1.996116  2.160462    66  0.282592 -2.219057  1.834042    66 -0.374322

In [285]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -3.500173 -0.679900    250 -1.282374
      data2 -3.138455  3.050345    250  0.038343
1     data1 -0.675212  0.022932    250 -0.288044
      data2 -2.710756  2.604857    250  0.020361
2     data1  0.024504  0.734481    250  0.356046
      data2 -2.523539  3.155617    250  0.060899
3     data1  0.734739  3.173246    250  1.348311
      data2 -4.401228  2.002017    250  0.120965

### | 🤔 `Example-1` - Filling Missing values with Group Sepcific values 

In [286]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s
s.fillna(s.mean())

0    0.451456
1    1.751414
2    0.451456
3   -0.057958
4    0.451456
5   -0.339088
dtype: float64

In [287]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data

Ohio          0.829436
New York      1.267807
Vermont       0.529816
Florida       0.399981
Oregon        1.261804
Nevada       -0.835541
California    1.910144
Idaho         1.527899
dtype: float64

In [288]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data
data.groupby(group_key).size()
data.groupby(group_key).count()
data.groupby(group_key).mean()

East    0.832408
West    1.585974
dtype: float64

In [289]:
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

East  Ohio          0.829436
      New York      1.267807
      Vermont       0.832408
      Florida       0.399981
West  Oregon        1.261804
      Nevada        1.585974
      California    1.910144
      Idaho         1.585974
dtype: float64

In [290]:
fill_values = {"East": 0.5, "West": -1}
def fill_func(group):
    return group.fillna(fill_values[group.name])

data.groupby(group_key).apply(fill_func)

East  Ohio          0.829436
      New York      1.267807
      Vermont       0.500000
      Florida       0.399981
West  Oregon        1.261804
      Nevada       -1.000000
      California    1.910144
      Idaho        -1.000000
dtype: float64

In [291]:
suits = ["H", "S", "C", "D"]  # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [292]:
deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [293]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

3S      3
4D      4
QS     10
3H      3
10H    10
dtype: int64

In [294]:
def get_suit(card):
    # last letter is suit
    return card[-1]

deck.groupby(get_suit).apply(draw, n=2)

C  AC     1
   JC    10
D  5D     5
   6D     6
H  8H     8
   3H     3
S  7S     7
   JS    10
dtype: int64

In [295]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

KC    10
4C     4
5D     5
JD    10
5H     5
8H     8
9S     9
AS     1
dtype: int64

### | 🤔 `Example-2` - Group Weighted Average and Correlation 

In [296]:
df = pd.DataFrame(
    {
        "category": ["a", "a", "a", "a", "b", "b", "b", "b"],
        "data": np.random.standard_normal(8),
        "weights": np.random.uniform(size=8)
    }
)
df

,category,data,weights
0,a,-0.498512,0.692456
1,a,0.033654,0.713197
2,a,0.422910,0.698001
3,a,0.244883,0.610433
4,b,-1.307162,0.478704
5,b,-0.139752,0.914225
6,b,1.106472,0.974823
7,b,-0.937327,0.685822


In [297]:
grouped = df.groupby("category")
def get_wavg(group):
    return np.average(group["data"], weights=group["weights"])

grouped.apply(get_wavg, include_groups=False)

category
a    0.045496
b   -0.104053
dtype: float64

---

## 4 | Pivot Tables and Cross-Tabulation
📌 `Note` - The default `aggregaion method` of pivot table is `mean`<br><br>
📌 `Note` - The columns which you provided in index and column attribute must be categorical

In [298]:
tips.head()
tips.pivot_table(
    index=["day", "smoker"],
    values=["size", "tip", "tip_pct", "total_bill"]
)

size       tip   tip_pct  total_bill
day  smoker                                          
Thur Yes     2.352941  3.030000  0.163863   19.190588
     No      2.488889  2.673778  0.160298   17.113111
Fri  Yes     2.066667  2.714000  0.174783   16.813333
     No      2.250000  2.812500  0.151650   18.420000
Sat  Yes     2.476190  2.875476  0.147906   21.276667
     No      2.555556  3.102889  0.158048   19.661778
Sun  Yes     2.578947  3.516842  0.187250   24.120000
     No      2.929825  3.167895  0.160113   20.506667

In [299]:
tips.pivot_table(
    index=["time", "day"], columns="smoker",
    values=["tip_pct", "size"]
)

size             tip_pct          
smoker            Yes        No       Yes        No
time   day                                         
Lunch  Thur  2.352941  2.500000  0.163863  0.160311
       Fri   1.833333  3.000000  0.188937  0.187735
Dinner Thur       NaN  2.000000       NaN  0.159744
       Fri   2.222222  2.000000  0.165347  0.139622
       Sat   2.476190  2.555556  0.147906  0.158048
       Sun   2.578947  2.929825  0.187250  0.160113

In [300]:
tips.pivot_table(
    index=["time", "day"], columns="smoker",
    values=["tip_pct", "size"], 
    margins=True # Means of Mean (both row and column wise)
)

size                       tip_pct                    
smoker            Yes        No       All       Yes        No       All
time   day                                                             
Lunch  Thur  2.352941  2.500000  2.459016  0.163863  0.160311  0.161301
       Fri   1.833333  3.000000  2.000000  0.188937  0.187735  0.188765
Dinner Thur       NaN  2.000000  2.000000       NaN  0.159744  0.159744
       Fri   2.222222  2.000000  2.166667  0.165347  0.139622  0.158916
       Sat   2.476190  2.555556  2.517241  0.147906  0.158048  0.153152
       Sun   2.578947  2.929825  2.842105  0.187250  0.160113  0.166897
All          2.408602  2.668874  2.569672  0.163196  0.159328  0.160803

In [301]:
tips.pivot_table(
    index=["time", "smoker"], 
    columns="day",
    values="tip_pct", 
    aggfunc=len, # Agg function
    margins=True
)

day            Thur   Fri   Sat   Sun  All
time   smoker                             
Lunch  Yes     17.0   6.0   NaN   NaN   23
       No      44.0   1.0   NaN   NaN   45
Dinner Yes      NaN   9.0  42.0  19.0   70
       No       1.0   3.0  45.0  57.0  106
All            62.0  19.0  87.0  76.0  244

In [302]:
tips.pivot_table(
    index=["time", "size", "smoker"], 
    columns="day",
    values="tip_pct", 
    fill_value=0
)

day                     Thur       Fri       Sat       Sun
time   size smoker                                        
Lunch  1    Yes     0.000000  0.223776  0.000000  0.000000
            No      0.181728  0.000000  0.000000  0.000000
       2    Yes     0.158843  0.181969  0.000000  0.000000
            No      0.166005  0.000000  0.000000  0.000000
       3    Yes     0.204952  0.000000  0.000000  0.000000
            No      0.084246  0.187735  0.000000  0.000000
       4    Yes     0.155410  0.000000  0.000000  0.000000
            No      0.138919  0.000000  0.000000  0.000000
       5    No      0.121389  0.000000  0.000000  0.000000
       6    No      0.173706  0.000000  0.000000  0.000000
Dinner 1    Yes     0.000000  0.000000  0.325733  0.000000
            No      0.000000  0.000000  0.137931  0.000000
       2    Yes     0.000000  0.171297  0.148668  0.207893
            No      0.159744  0.139622  0.162705  0.168859
       3    Yes     0.000000  0.000000  0.144995  0.152660
            No      0.000000  0.000000  0.154661  0.152663
       4    Yes     0.000000  0.117750  0.124515  0.193370
            No      0.000000  0.000000  0.150096  0.148143
       5    Yes     0.000000  0.000000  0.106572  0.065660
            No      0.000000  0.000000  0.000000  0.206928
       6    No      0.000000  0.000000  0.000000  0.103799

### | Cross Tabulations

In [303]:
from io import StringIO
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep="\s+")

In [304]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


📌 `Note` - The agg function is `frequency`

In [305]:
pd.crosstab(index=data["Nationality"], columns=data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [306]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

smoker       Yes   No  All
time   day                
Lunch  Thur   17   44   61
       Fri     6    1    7
Dinner Thur    0    1    1
       Fri     9    3   12
       Sat    42   45   87
       Sun    19   57   76
All           93  151  244